<a href="https://colab.research.google.com/github/bryandaetz1/SB_County_COVID-19_Data/blob/master/SB_County_Public_Health_Web_Scraper_7_6_20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [50]:
#specify URL/web page to scrape
url = 'https://publichealthsbc.org/status-reports/'

#create user-agent (Windows 10 with Google Chrome)
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '\
'AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.03987.149 '\
'Safara/537.36'

headers = {'User-Agent': user_agent}

response = requests.get(url, headers = headers).text

In [51]:
#parse the HTML from the URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(response, 'html.parser')

In [52]:
#grabbing page title to verify that we were able to successfully get requests from webpage 
soup.title

<title>Status Reports – County of Santa Barbara</title>

In [53]:
#after inspecting the page, it looks like all the tables that contain data we want have id = 'stats'
tables = soup.find_all('div', {'class': 'elementor-text-editor elementor-clearfix'})
len(tables)

8

In [54]:
#trying to create function to convert html table to pandas dataframe, had to break this into two functions

def TableToList(table):
  #getting contents of table and converting the text into a list
  raw = list(table.find_all('td'))
  text = []
  for i in raw:
    text.append(i.text)
  return text

def ListToDf(list, no_columns):
  # calculating number of rows for dataframe
  rows = int(len(list) / no_columns)

  # converting list to numpy array to reshape
  array = np.array(list)
  array = np.reshape(array, (rows, no_columns))

  #converting array to pandas dataframe
  df = pd.DataFrame(array[1:], columns = array[0])
  return df


In [55]:
table1 = TableToList(tables[0])
#table1

In [56]:
current_stats = ListToDf(table1, 6)
current_stats

,Geographic Area as of 7/5/2020,DailyCases,Total Confirmed Cases,Recovered by Region,Active Cases by Region,Number of Deaths
0,SOUTH COUNTY UNINCORPORATED AREA includes comm...,0,64,53,10,0
1,CITY OF SANTA BARBARA and the unincorporated a...,8,368,308,58,3
2,CITY OF GOLETA,0,68,59,7,2
3,COMMUNITY OF ISLA VISTA,2,20,19,1,0
4,UNINCORPORATED AREA OF THE GOLETA VALLEY AND G...,1,55,42,12,1
5,SANTA YNEZ VALLEY including the Cities of Solv...,0,25,23,2,0
6,CITY OF LOMPOC and the communities of Mission ...,5,177,159,14,4
7,FEDERAL PRISON IN LOMPOC,0,994,991,0,3
8,CITY OF SANTA MARIA,42,1392,1214,164,15
9,COMMUNITY OF ORCUTT,7,93,82,10,0


In [57]:
table2 = TableToList(tables[1])
table2 = table2[1:]
#table2

In [58]:
cases_by_age = ListToDf(table2, 5)
cases_by_age

,Age Range,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,0 – 17,6,226,0,226
1,18 – 29,25,705,134,839
2,30 – 49,28,916,526,1442
3,50 – 69,11,500,302,802
4,70+,6,143,32,175
5,Age Suppressed,0,—,—,—


In [59]:
table3 = TableToList(tables[2])
table3 = table3[1:]
#table3

In [60]:
cases_by_sex = ListToDf(table3, 5)
cases_by_sex.columns.values[0] = 'Sex'
cases_by_sex

,Sex,Daily,Community,Federal Prisonin Lompoc,Total(Community& Prison)
0,Female,44,1200,2,1202
1,Male,31,1273,979,2252
2,Unknown,1,17,13,30
3,Pending,0,0,0,0


In [61]:
table4 = TableToList(tables[3])
#table4

In [62]:
testing_status = ListToDf(table4, 2)
testing_status

,Testing Status,Total
0,Positive,3484
1,Negative,45496
2,Inconclusive,145
3,Total Tests,49125


In [63]:
table5 = TableToList(tables[4])
#table5

In [64]:
high_risk = ListToDf(table5[:4],2)
high_risk

,High Risk,Total
0,Health Care Workers (HCW),157


In [65]:
transmission_method = ListToDf(table5[4:], 2)
transmission_method

,Transmission Method,Total
0,Community Close Contact Transmission,741
1,Persons Incarcerated at Federal Prison in Lomp...,994
2,Travel Transmission,12
3,Community Transmission,546
4,Under Investigation,1193


In [66]:
table6 = TableToList(tables[5])

#ran into an issue convering list to dataframe because website had missing values after 'Recovering in Home' and 'NO LONGER REPORTING', using a rough workaround for now to add "--" to the places where values were missing
table6.insert(13, '--')
table6.insert(15, '--')
#table6

In [67]:
recovery_status = ListToDf(table6, 4)
recovery_status

,RECOVERY STATUS,Community,Federal Prison in Lompoc,Total(Community & Prison)
0,Active Cases,305,0,305
1,Recovered Cases,2159,991,3150
2,Recovering at Home,--,NO LONGER REPORTING,--
3,Recovering in Hospital,71,—,—
4,Recovering in ICU,21,—,—
5,Pending Information,—,—,—


In [68]:
table7 = TableToList(tables[6])
#table7

In [69]:
ethnicity = ListToDf(table7, 5)
ethnicity

,RACE/ETHNICITY,Daily,Community,Federal Prison in Lompoc,Total(Community & Prison)
0,Hispanic/Latino,37,1234,377,1611
1,"White, Non-Hispanic",4,189,383,571
2,"Asian, Non-Hispanic",0,12,51,63
3,"Black/African American, Non-Hispanic",1,9,147,156
4,"American Indian/Native Alaskan, Non-Hispanic",0,5,28,33
5,"Native Hawaiian or Pacific Islander, Non-Hispanic",0,2,1,3
6,"Multiracial, Non-Hispanic",0,5,0,5
7,"Other, Non-Hispanic",2,21,0,21
8,Unknown Ethnicity,6,108,0,108
9,"Unknown Race, Non-Hispanic",1,14,0,14
